In [2]:
import cv2
import numpy as np
import os
import csv 

from utils import *

In [ ]:
"Bước 1: Lấy mẫu video, frame_distance là tham số khoảng cách giữa 2 lần lấy mẫu liên tiếp"

video_path = "DJI_0044.mp4"
frame_distance = 24
output_folder = "v44_24"
extracted_frames = extract_frames(video_path, frame_distance, output_folder)

In [3]:
"""
Bước 2: Từ các khung hình đã lấy mẫu từ trước, tính toán ma trận H giữa 2 khung liên tiếp
Sử dụng thuật toán akaze dùng hàm "findH_akaze"
Sử dụng thuật toán sift dùng hàm "findH_sift"
Sử dụng thuật toán orb dùng hàm "findH_orb"
"""

folder_path = "v44_384"
image_files = os.listdir(folder_path)
image_files.sort()
image_0 = cv2.imread(os.path.join(folder_path, image_files[0]))
H = []

for i, img_file in enumerate(image_files[1:], 1):
    next_image = cv2.imread(os.path.join(folder_path, img_file))
    homo = findH_akaze(image_0, next_image)
    H.append(homo)
    image_0 = next_image

csv_file = "homographies_v44_384_akaze.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for i, homo in enumerate(H):
        writer.writerow([f"H_{i}"])
        for row in homo:
            writer.writerow(row)
    print(f"Homographies have been saved to {csv_file}")

Homographies have been saved to D:/Xu ly anh/thucnghiem/homographies_v44_384_akaze.csv


In [15]:
"""
Đọc ma trận H từ file đã lưu
Tính toán kích thước ảnh đầu ra
Bắt đầu ghép ảnh và xuất file
"""


csv_file = "D:/Xu ly anh/thucnghiem/homographies_v44_384_akaze.csv"
H = read_homographies_from_csv(csv_file)

folder_path = "D:/Xu ly anh/thucnghiem/v44_384"
image_files = os.listdir(folder_path)
image_files.sort()
images = [cv2.imread(os.path.join(folder_path, img_file)) for img_file in image_files]

output_width, output_height, min_x, min_y = calculate_output_size(images, H)
output_shape = (output_width, output_height)
print(output_shape,min_x,min_y)

result = np.zeros((output_shape[1], output_shape[0], 3), dtype=np.uint8)

for i, image in enumerate(images):
    warped_image = warp_image_to_zero(image, H, i, output_shape, min_x, min_y)
    result = np.maximum(result, warped_image)
    cv2.imwrite("v44_384.jpg", result)

(4820, 11084) -3406.9653 -4901.0654
